# Plotting for experiments 1 and 2


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import graphlearning as gl
import graphlearning.active_learning as al

import batch_active_learning as bal
import utils

from matplotlib.ticker import AutoLocator

In [ ]:
#Non-default Parameters

acq_fun_list = ['uc']
save_path = "Experiment Results/Experiment 1/"

In [ ]:
def experiment1_plotter(x_dict, y_dict, dataset, include_sota=False):
    fig, ax1 = plt.subplots()
    
    dataset_size = utils.SAR_DATASET_SIZE_DICT[dataset]

    ax1.set_xlabel('Number of Labeled Points')
    ax1.set_ylabel('Accuracy (%)')
    for i in range(len(utils.AL_METHODS)):
        this_key = utils.AL_METHODS[i]
        ax1.plot(x_dict[this_key], y_dict[this_key], label=utils.AL_METHOD_NAMES[i])
    ax1.tick_params(axis='x')
    
    #Add SoTA
    if dataset != 'mstar' and include_sota:
        if dataset == 'open_sar_ship':
            sota_val = 78.15
        elif dataset == 'fusar':
            sota_val = 86.69
        ax1.plot(x_dict['local_max'], sota_val * np.ones_like(x_dict['local_max']), label='SoTA', linestyle='--')
    
    ax1.legend()
    
    ax2 = ax1.twiny() 
    
    #Get percentages on top
    ax2.set_xlabel('Percent of Dataset Labeled')
    ax2.plot(100 * x_dict['local_max'] / (1.0 * dataset_size), y_dict['local_max'])
    ax2.tick_params(axis='x')
    ax2.xaxis.set_major_locator(AutoLocator())
    fig.tight_layout()
    
    file_path = save_path + dataset + '_detailed_plot'
    if include_sota:
        file_path += '_sota'
    file_path += '.png'
    
    plt.savefig(file_path, bbox_inches='tight')
    plt.show()
    return